In [1]:
# pip install PyPDF2

In [2]:
# pip install PyPDF2

In [3]:
# pip install selenium

In [4]:
# Import libraries
import PyPDF2 as pydf
import re
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import os
import time

In [5]:
# Align driver
DRIVER_PATH = '\\Users\\Dave\\Desktop\\Python\\chromedriver\\chromedriver.exe'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)

## <font color='teal'>Retrieve PDFs</font>

In [6]:
# Download location
primary_bills_download_folder = 'C:\\Users\\Dave\\Desktop\\Job Apps\\Legislative Assistant, Senator Marcus\\Marcus Primary Bills'

In [22]:
# Update Chrome settings to allow for pdf downloads
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    'download.default_directory': primary_bills_download_folder, # Change default directory for downloads
    'download.prompt_for_download': False, # To auto download the file
    'download.directory_upgrade': True,
    'plugins.always_open_pdf_externally': True # It will not show PDF directly in chrome
})
driver = webdriver.Chrome(executable_path=DRIVER_PATH, options=options)

In [23]:
ncleg_search = 'https://www.ncleg.gov/Search/BillText'
search_keys = 'Marcus'
session_keys = '2019-2020 Session'

In [24]:
search_dropdown_xpath = '//*[@id="gsSearch"]'
search_input_xpath = '//*[@id="sSearchText"]'
session_dropdown_xpath = '//*[@id="sSession_chosen"]'
session_autocomplete_xpath = '//*[@id="sSession_chosen"]/div/div/input'
search_button_xpath = '//*[@id="billTextSearchForm"]/div/div[6]/button'

In [25]:
# Navigate to ncleg search site
driver.get(ncleg_search)

In [26]:
# Fill in Search options

# Open the Search dropdown (this dropdown should
# already be open, so no need to click
# # driver.find_element_by_xpath(search_dropdown_xpath).click()

# Send search info
driver.find_element_by_xpath(search_input_xpath).clear() # First clear search
driver.find_element_by_xpath(search_input_xpath).send_keys(search_keys) # Then send keywords

# Click to expand dropdown, filter to desired sesseion
driver.find_element_by_xpath(session_dropdown_xpath).click()
driver.find_element_by_xpath(session_autocomplete_xpath).send_keys(session_keys)

# Having filtered to sought option, select it
active_element = driver.switch_to_active_element()
active_element.send_keys(Keys.ENTER)

# Press the Search button
driver.find_element_by_xpath(search_button_xpath).click()

C:\Users\Dave\anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: use driver.switch_to.active_element instead
  app.launch_new_instance()


In [ ]:
def retrieveBills():
    # Loop to retrieve players by href
    bills_list = []
    elements = driver.find_elements_by_xpath('//a[contains(@href, "/BillLookUp/")]')
    for element in elements:
        bills_list.append(element)
    num_bills = len(bills_list)
    print('Identified {} bills to acquire.'.format(num_bills))
    
    for i in list(range(0, num_bills)):
        # Need to rerun bills_list each time in order to refresh elements
        bills_list = []
        elements = driver.find_elements_by_xpath('//a[contains(@href, "/BillLookUp/")]')
        for element in elements:
            bills_list.append(element)
        
        # Navigate to bill specifics site
        bills_list[i].click()
        
        ### Retrieve bill information, then download
        
        # Retrieve bill number and name
        bill_number_xpath = '/html/body/div[1]/div/main/div[1]/div[3]'
        bill_number = driver.find_element_by_xpath(bill_number_xpath).text
        bill_name_xpath = '/html/body/div[1]/div/main/div[2]/div[1]/a'
        bill_name = driver.find_element_by_xpath(bill_name_xpath).text

        # Retrieve bill sponsors, clean and transform
        bill_sponsors_xpath = '/html/body/div[1]/div/main/div[3]/div[2]/div/div[4]'
        bill_sponsors = driver.find_element_by_xpath(bill_sponsors_xpath).text
        bill_sponsors_cleaned = bill_sponsors.replace('\n', ';').replace('(Primary)', '')
        bill_sponsors_individual = bill_sponsors_cleaned.split(';')
        bill_sponsors_individual_cleaned = []
        for ind in bill_sponsors_individual:
            bill_sponsors_individual_cleaned.append(ind.strip())

        # Check which sponsor(s) is primary
        bill_sponsors_dict = {}
        for sp in bill_sponsors_individual_cleaned:
            if bill_sponsors.find(sp) < bill_sponsors.find('Primary'):
                bill_sponsors_dict[sp] = {'Primary':True}
            else:
                bill_sponsors_dict[sp] = {'Primary':False}

        # Retrieve bill attributes
        bill_attributes_xpath = '/html/body/div[1]/div/main/div[3]/div[2]/div/div[6]'
        bill_attributes = driver.find_element_by_xpath(bill_attributes_xpath).text
        if bill_attributes.split(';')[0] == 'Public':
            bill_pub_loc = 'Public'
        elif bill_attributes.split(';')[0] == 'Local':
            bill_pub_loc = 'Local'
        else:
            bill_pub_loc = None

        # Retrieve bill keywords
        bill_keywords_xpath = '/html/body/div[1]/div/main/div[3]/div[2]/div/div[12]'
        bill_keywords = driver.find_element_by_xpath(bill_keywords_xpath).text

        # Store information about bill in dict
        bill_dict = {}
        bill_dict[bill_number] = {'Name':bill_name,
                                  'Sponsors':bill_sponsors_dict,
                                  'Breadth':bill_pub_loc,
                                  'Keywords':bill_keywords}
        
        # Click to download pdf
        driver.find_element_by_xpath(bill_name_xpath).click()
        
        

In [40]:
# # Loop to retrieve players by href
# bills_list = []
# elements = driver.find_elements_by_xpath('//a[contains(@href, "/BillLookUp/")]')
# for element in elements:
#     bills_list.append(element)
# print(len(bills_list))

In [41]:
# # Navigate to bill specifics site
# bills_list[1].click()

In [42]:
# # Retrieve bill number and name
# bill_number_xpath = '/html/body/div[1]/div/main/div[1]/div[3]'
# bill_number = driver.find_element_by_xpath(bill_number_xpath).text
# bill_name_xpath = '/html/body/div[1]/div/main/div[2]/div[1]/a'
# bill_name = driver.find_element_by_xpath(bill_name_xpath).text

# # Retrieve bill sponsors, clean and transform
# bill_sponsors_xpath = '/html/body/div[1]/div/main/div[3]/div[2]/div/div[4]'
# bill_sponsors = driver.find_element_by_xpath(bill_sponsors_xpath).text
# bill_sponsors_cleaned = bill_sponsors.replace('\n', ';').replace('(Primary)', '')
# bill_sponsors_individual = bill_sponsors_cleaned.split(';')
# bill_sponsors_individual_cleaned = []
# for ind in bill_sponsors_individual:
#     bill_sponsors_individual_cleaned.append(ind.strip())

# # Check which sponsor(s) is primary
# bill_sponsors_dict = {}
# for sp in bill_sponsors_individual_cleaned:
#     if bill_sponsors.find(sp) < bill_sponsors.find('Primary'):
#         bill_sponsors_dict[sp] = {'Primary':True}
#     else:
#         bill_sponsors_dict[sp] = {'Primary':False}
        
# # Retrieve bill attributes
# bill_attributes_xpath = '/html/body/div[1]/div/main/div[3]/div[2]/div/div[6]'
# bill_attributes = driver.find_element_by_xpath(bill_attributes_xpath).text
# if bill_attributes.split(';')[0] == 'Public':
#     bill_pub_loc = 'Public'
# elif bill_attributes.split(';')[0] == 'Local':
#     bill_pub_loc = 'Local'
# else:
#     bill_pub_loc = None

# # Retrieve bill keywords
# bill_keywords_xpath = '/html/body/div[1]/div/main/div[3]/div[2]/div/div[12]'
# bill_keywords = driver.find_element_by_xpath(bill_keywords_xpath).text

# # Store information about bill in dict
# bill_dict = {}
# bill_dict[bill_number] = {'Name':bill_name,
#                           'Sponsors':bill_sponsors_dict,
#                           'Breadth':bill_pub_loc,
#                           'Keywords':bill_keywords}

In [38]:
# Click to download pdf
driver.find_element_by_xpath(bill_name_xpath).click()

In [39]:
# Navigate back to bills list page
driver.back()

In [ ]:
# bill_dict['Senate Bill 861'].keys()

In [ ]:
# bill_dict['Senate Bill 861']['Sponsors']['Marcus']['Primary']

In [43]:
# Walk os to retrieve files from Downloads folder
onlyfiles = [f for f in os.listdir(primary_bills_download_folder) if os.path.isfile(os.path.join(primary_bills_download_folder, f))]

In [47]:
target_file = []
bill_num_as_str = bill_number.split(' ')[-1]
for file in onlyfiles:
    if bill_num_as_str in file:
        target_file.append(file)

if len(target_file) > 1:
    print('Error, too many matching files...')
elif len(target_file) == 0:
    print('Error, zero matching files')
elif len(target_file) == 1:
    print('Run function here...')

Run function here...


In [51]:
target_full_filepath = primary_bills_download_folder + '\\' + target_file[0]

In [52]:
target_full_filepath

'C:\\Users\\Dave\\Desktop\\Job Apps\\Legislative Assistant, Senator Marcus\\Marcus Primary Bills\\S797v1.pdf'

## <font color='teal'>Extract from PDF</font>

In [ ]:
# Set path of pdf to extract
pdf_filepath = 'C:\\Users\\Dave\\Desktop\\Job Apps\\Legislative Assistant, Senator Marcus\\Marcus Primary Bills\\Election Protection Covid-19 Pandemic.pdf'

In [ ]:
# Create pdf object and retrieve page count 
pdf = pydf.PdfFileReader(open(pdf_filepath, 'rb'))
num_pages = pdf.getNumPages()

In [ ]:
# Function to extract text from pdf, page by page, and write to dict
def text_extractor(path):
    pages_dict = {}
    with open(pdf_filepath, 'rb') as file:
        pdf = pydf.PdfFileReader(file)
        for i in list(range(0, num_pages)):
            page = pdf.getPage(i)
            pages_dict[i+1] = page.extractText()
    return pages_dict;

In [ ]:
# Run function
pages_dict = text_extractor(pdf_filepath)

In [ ]:
# Function to concatenate text from pages to single string
def page_compiler(pages_dict):
    full_text = ''
    for page in pages_dict.keys():
        full_text = full_text + pages_dict[page]
    return full_text;

In [ ]:
# Run function
full_text = page_compiler(pages_dict)

In [ ]:
words_to_skip_list = ['a', 'the', 'an', 'of', 'be', 'to', 'when', 'where', 'which',
                      'north', 'carolina', 'session']

In [ ]:
test = re.findall(r'\w+', full_text)

In [ ]:
test_unique = list(set(test))

In [ ]:
test_unique.sort(reverse=False)

In [ ]:
word_counts_dict = {}
for word in test_unique:
    if word in words_to_skip_list:
        print('Skipping {}'.format(word))
    else:
        count = full_text.count(word)
        word_counts_dict[word] = count

In [ ]:
test_df = pd.DataFrame.from_dict(word_counts_dict, orient='index', columns=['Count'])

In [ ]:
test_df.sort_values('Count', ascending=False, inplace=True)

In [ ]:
test_df[0:100]

In [ ]:
# # Function to retrieve unique words in pdf, build counts of each word
# def word_counter(pdf_string):
    

In [ ]:
test_string = "Tutorials point is a learning platform"
# original string
print ("The original string is : " + test_string)
# using regex (findall()) function
res = len(re.findall(r'\w+', test_string))
# total no of words
print ("The number of words in string are : " + str(res))

In [ ]:
re.findall(r'\w+', test_string)